In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os
import contextily as ctx
import geopandas as gpd
from shapely.geometry import Point

# Load data
positions = pd.read_csv('../data/vessel_positions.csv')
events = pd.read_csv('../data/simulated_vessel_proximity_events.csv')

# Convert timestamps
positions['datetime'] = pd.to_datetime(positions['t'], unit='ms')
events['datetime'] = pd.to_datetime(events['t'], unit='ms')

# Select one proximity event
event = events.iloc[0]
vid1, vid2 = event['vessel_id1'], event['vessel_id2']
event_time, event_lat, event_lon = event['datetime'], event['lat'], event['lon']

# Filter vessel positions
v1 = positions[positions['vessel_id'] == vid1].sort_values('datetime')
v2 = positions[positions['vessel_id'] == vid2].sort_values('datetime')

# Allow a ±1 minute window for matching timestamps
window = pd.Timedelta(minutes=1)
timestamps = []

for t1 in v1['datetime']:
    for t2 in v2['datetime']:
        if abs(t1 - t2) <= window:
            timestamps.append(t1)

timestamps = sorted(set(timestamps))

# Convert to Web Mercator (for basemap)
def to_webmercator(df):
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326")
    return gdf.to_crs("EPSG:3857")

v1_gdf = to_webmercator(v1)
v2_gdf = to_webmercator(v2)
event_point = to_webmercator(pd.DataFrame({'lat':[event_lat], 'lon':[event_lon]})).geometry.iloc[0]

# Create frames directory
os.makedirs("frames", exist_ok=True)
filenames = []

print("📸 Generating frames...")

for i, ts in enumerate(timestamps):
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.set_title(f"Vessel Proximity Event – {ts.strftime('%H:%M:%S')}", fontsize=12)

    # Get data up to timestamp
    v1_now = v1_gdf[v1_gdf['datetime'] <= ts]
    v2_now = v2_gdf[v2_gdf['datetime'] <= ts]

    if v1_now.empty or v2_now.empty:
        continue

    # Plot trajectories
    v1_coords = list(zip(v1_now.geometry.x, v1_now.geometry.y))
    v2_coords = list(zip(v2_now.geometry.x, v2_now.geometry.y))

    ax.plot(*zip(*v1_coords), color='blue', lw=1, label=f'Vessel {vid1} Trajectory')
    ax.plot(*zip(*v2_coords), color='green', lw=1, label=f'Vessel {vid2} Trajectory')

    # Plot current positions with vessel IDs and speeds
    v1_x, v1_y, v1_speed = v1_now.geometry.iloc[-1].x, v1_now.geometry.iloc[-1].y, v1_now['speed'].iloc[-1]
    v2_x, v2_y, v2_speed = v2_now.geometry.iloc[-1].x, v2_now.geometry.iloc[-1].y, v2_now['speed'].iloc[-1]

    ax.plot(v1_x, v1_y, 'bo')
    ax.text(v1_x, v1_y, f"{vid1}\nSpeed: {v1_speed} km/h", fontsize=9, color='blue', ha='left', va='bottom')

    ax.plot(v2_x, v2_y, 'go')
    ax.text(v2_x, v2_y, f"{vid2}\nSpeed: {v2_speed} km/h", fontsize=9, color='green', ha='left', va='bottom')

    # Show collision point
    if ts >= event_time:
        ax.text(event_point.x, event_point.y, "💥", fontsize=18, ha='center', va='center')

    # Add basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=v1_gdf.crs.to_string())

    # Map settings
    ax.set_xlim(min(v1_gdf.geometry.x.min(), v2_gdf.geometry.x.min()) - 5000,
                max(v1_gdf.geometry.x.max(), v2_gdf.geometry.x.max()) + 5000)
    ax.set_ylim(min(v1_gdf.geometry.y.min(), v2_gdf.geometry.y.min()) - 5000,
                max(v1_gdf.geometry.y.max(), v2_gdf.geometry.y.max()) + 5000)

    ax.set_axis_off()
    ax.legend()

    # Save frame
    fname = f"frames/frame_{i:03d}.png"
    plt.savefig(fname, bbox_inches='tight', dpi=100)
    filenames.append(fname)
    plt.close()

# Build slow, looping GIF
print("🎞️ Creating GIF...")
if filenames:
    with imageio.get_writer("collision_course_11_12.gif", mode="I", duration=2.5, loop=0) as writer:
        for f in filenames:
            writer.append_data(imageio.imread(f))
else:
    print("❌ No frames to include in GIF.")

# Clean up frames
#for f in filenames:
#    os.remove(f)

print("✅ GIF saved as 'collision_course_11_12.gif'")